ライブラリをインポートする

In [1]:
!python -m pip install semantic-kernel==0.5.1.dev0
!python -m pip install slack-sdk load_dotenv
!python -m pip install guidance


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


コンフィグを読み取る

In [2]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    OpenAIChatCompletion
)

kernel = sk.Kernel()

useAzureOpenAI = True

if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    azure_chat_service = AzureChatCompletion(
        deployment_name=deployment,
        api_key=api_key,
        endpoint=endpoint
    )
    kernel.add_chat_service("chat_completion", azure_chat_service)
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    openai_chat_service = OpenAIChatCompletion(
        ai_model_id="gpt-3.5-turbo",
        api_key=api_key,
        org_id=org_id
    )
    kernel.add_chat_service("chat-gpt", openai_chat_service)

In [3]:
from plugins.SlackPlugin.slack_plugin import SlackPlugin
plugins_directory = "plugins"
messageFunctions = kernel.import_semantic_plugin_from_directory(plugins_directory, "MessagePlugin")

slack_plugin = kernel.import_plugin(SlackPlugin(), "SlackPlugin")

DEBUG:semantic_kernel.template_engine.prompt_template_engine:Extracting blocks from template: WRITE ONE or TWO LINE MESSAGES BASED ON THE TOPIC BELOW

JOKE MUST BE:
- IN JAPANESE
- G RATED
- WORKPLACE/FAMILY SAFE
NO SEXISM, RACISM OR OTHER BIAS/BIGOTRY

Incorporate the style suggestion, if provided: {{$style}}
+++++

{{$subject}}
+++++

DEBUG:semantic_kernel.kernel:Importing plugin SlackPlugin
DEBUG:semantic_kernel.kernel:Methods imported: 1


ユーザとやりとりをして、Planを作成する

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

import guidance
from guidance import system, user, assistant, gen

buddy = guidance.models.AzureOpenAIChat(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

skills = [
    {"name": "GenerateMessageJa", "description": "Compose a short message in Japanese based on the input subject. Execute this function before sending a message anywhere."},
    {"name": "SendMessage", "description": "Send formatted messages to Slack."},
]

@guidance
def experts(lm, input, skills):
    with system():
        lm += f"""\
        # 前提条件
            - あなたはユーザの入力に基づいて「実行計画」を作成する優秀なプランナーです。
            - 実行計画はユーザの入力を達成するために必要な作業内容を分解および順序づけしたものです。
            - 実行計画には関数の利用を含みます。
            - 実行計画で利用できる関数は「使用可能な関数」で定義された関数のみです。
        """

    with user():
        lm += f"""
        # ゴール
            - ユーザの入力から実行計画を作成する
            - 出力形式は以下のようなJSONフォーマットであること
                ```
                {{
                    "user_input": "ユーザの入力",
                    "best_plan": [実行する関数名のリスト],
                    "description": "実行する関数名とその関数を利用する目的の説明"
                }}
                ```
        # 実行のプロセス
            1. ユーザの入力を受け取り、実行計画を3つ作成する
            2. 最も有効な実行計画が1位となるようにランク付けする
            3. 1位の実行計画をJSONフォーマットに変換する
            4. JSONフォーマットに変数した実行計画のみを出力する
        # 使用可能な関数
        """

        for i in range(len(skills)):
            lm += f"""
                - {skills[i]["name"]}: {skills[i]["description"]}
            """

    with assistant():
        lm += f"""\
            はい。ユーザの入力に基づいて実行計画を作成し、JSONのみを出力します。
            また、実行計画の作成には使用可能な関数のみを使用します。
        """
    
    with user():
        lm += f"""\
            ユーザの入力:{input}
            必ずJSON形式の部分のみを出力してください。
        """
    
    with assistant():
        lm += gen(name='answer', max_tokens=500)

    return lm

result = None

while True:
    user_input = input("なにを依頼したいですか？特に依頼がなければ'end'と入力してください。:")
    if user_input == "end":
        break
    else:
        result = buddy + experts(input=user_input, skills=skills)

In [ ]:
print(result['answer'])

SKにPlanを作成させる

In [5]:
from semantic_kernel.planning.basic_planner import BasicPlanner
from semantic_kernel.planning.stepwise_planner import StepwisePlanner
from semantic_kernel.planning.stepwise_planner.stepwise_planner_config import (
    StepwisePlannerConfig,
)

plan_base = result["answer"]

print(f"### Plan ###\n{plan_base}")

# goal = "Slackにいるメンバーに、出社したことを知らせてください。"
# step_planner = StepwisePlanner(kernel, StepwisePlannerConfig(max_iterations=10, min_iteration_time_ms=1000))
# step_plan = step_planner.create_plan(goal=goal)
# result = await step_plan.invoke()
# print(result)

basic_planner = BasicPlanner()
basic_plan = await basic_planner.create_plan(plan_base, kernel)

print(basic_plan.generated_plan)

DEBUG:semantic_kernel.template_engine.prompt_template_engine:Extracting blocks from template: 
You are a planner for the Semantic Kernel.
Your job is to create a properly formatted JSON plan step by step, to satisfy the goal given.
Create a list of subtasks based off the [GOAL] provided.
Each subtask must be from within the [AVAILABLE FUNCTIONS] list. Do not use any functions that are not in the list.
Base your decisions on which functions to use from the description and the name of the function.
Sometimes, a function may take arguments. Provide them if necessary.
The plan should be as short as possible.
For example:

[AVAILABLE FUNCTIONS]
EmailConnector.LookupContactEmail
description: looks up the a contact and retrieves their email address
args:
- name: the name to look up

WriterPlugin.EmailTo
description: email the input text to a recipient
args:
- input: the text to email
- recipient: the recipient's email address. Multiple addresses may be included if separated by ';'.

WriterPlu

### Plan ###
{
    "user_input": "出社したことを知らせてください。メッセージは明るい雰囲気にしてください。",
    "best_plan": ["GenerateMessageJa", "SendMessage"],
    "description": "出社したことを知らせるために、まずは明るい雰囲気のメッセージを生成し、それをSlackに送信します。"
}


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Cache-Control', b'no-cache, must-revalidate'), (b'Content-Length', b'641'), (b'Content-Type', b'application/json'), (b'access-control-allow-origin', b'*'), (b'apim-request-id', b'9f9202e8-7210-4b83-a74d-7ecaf5ade743'), (b'Strict-Transport-Security', b'max-age=31536000; includeSubDomains; preload'), (b'x-content-type-options', b'nosniff'), (b'x-ms-region', b'Japan East'), (b'x-ratelimit-remaining-requests', b'119'), (b'x-ratelimit-remaining-tokens', b'118500'), (b'x-accel-buffering', b'no'), (b'x-request-id', b'594b2c83-10eb-492f-a63a-7beee3e8de7f'), (b'x-ms-client-request-id', b'9f9202e8-7210-4b83-a74d-7ecaf5ade743'), (b'azureml-model-session', b'turbo-0613-acb7e666'), (b'Date', b'Tue, 13 Feb 2024 02:28:33 GMT')])
INFO:httpx:HTTP Request: POST https://buddy-playground.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
DEBUG:http

{
    "input": "出社したことを知らせてください。メッセージは明るい雰囲気にしてください。",
    "subtasks": [
        {"function": "MessagePlugin.GenerateMessageJa", "args": {"subject": "出社のお知らせ", "style": "明るい"}},
        {"function": "SlackPlugin.SendMessage"}
    ]
}


In [6]:
results = await basic_planner.execute_plan(basic_plan, kernel)
print(results)

DEBUG:semantic_kernel.template_engine.prompt_template_engine:Rendering string template: WRITE ONE or TWO LINE MESSAGES BASED ON THE TOPIC BELOW

JOKE MUST BE:
- IN JAPANESE
- G RATED
- WORKPLACE/FAMILY SAFE
NO SEXISM, RACISM OR OTHER BIAS/BIGOTRY

Incorporate the style suggestion, if provided: {{$style}}
+++++

{{$subject}}
+++++

DEBUG:semantic_kernel.template_engine.prompt_template_engine:Extracting blocks from template: WRITE ONE or TWO LINE MESSAGES BASED ON THE TOPIC BELOW

JOKE MUST BE:
- IN JAPANESE
- G RATED
- WORKPLACE/FAMILY SAFE
NO SEXISM, RACISM OR OTHER BIAS/BIGOTRY

Incorporate the style suggestion, if provided: {{$style}}
+++++

{{$subject}}
+++++

DEBUG:semantic_kernel.template_engine.prompt_template_engine:Rendering list of 5 blocks
DEBUG:semantic_kernel.template_engine.prompt_template_engine:Rendered prompt: WRITE ONE or TWO LINE MESSAGES BASED ON THE TOPIC BELOW

JOKE MUST BE:
- IN JAPANESE
- G RATED
- WORKPLACE/FAMILY SAFE
NO SEXISM, RACISM OR OTHER BIAS/BIGOTRY

In

Message sent!
